In [1]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [2]:
# !rm -f minsearch.py
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [3]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [18]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=3
    )

    return results

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   36G   65G  36% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   36G   65G  36% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    2.0G  132M  1.8G   7% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  5.4M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [7]:
from huggingface_hub import login

In [8]:
login(token=os.environ['HF_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /run/cache/token
Login successful


In [9]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import pipeline

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [11]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [14]:
generator("A list of colors: red, blue", max_length=500, truncation=True, temperature=0.7, top_p=0.95, num_return_sequences=1)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'A list of colors: red, blue, green, yellow, orange, purple, pink, black, white, brown, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, beige, cream, ivory, tan, gray, silver, gold, tan, 

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"{doc['question']}\n{doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = generator(prompt, max_length=500, temperature=0.7, top_p=0.95, num_return_sequences=1)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()

In [19]:
rag("I just discovered the course. Can I still join it?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Course - How can I join the course?\nYou can join the course by registering here.\n\nCourse - How can I join the course if I don't have a credit card?\nYou can join the course by registering here.\n\nCourse - How can I join the course if I don't have a PayPal account?\nYou can join the course by registering here.\n\nCourse - How can I join the course if I don't have a Stripe account?\nYou can join the course by registering here.\n\nCourse - How can I join the course if I don't have a Google account?\nYou can join the course by registering here.\n\nCourse - How can I join the course if I don't have a Facebook account?\nYou can join the course by registering here.\n\nCourse - How can I join the course if I don'"

In [20]:
print(_)

Course - How can I join the course?
You can join the course by registering here.

Course - How can I join the course if I don't have a credit card?
You can join the course by registering here.

Course - How can I join the course if I don't have a PayPal account?
You can join the course by registering here.

Course - How can I join the course if I don't have a Stripe account?
You can join the course by registering here.

Course - How can I join the course if I don't have a Google account?
You can join the course by registering here.

Course - How can I join the course if I don't have a Facebook account?
You can join the course by registering here.

Course - How can I join the course if I don'
